文本分类实例


导包

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

加载数据

In [4]:
import pandas as pd

data = pd.read_csv('ChnSentiCorp_htl_all.csv')
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [5]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


创建Dataset

In [7]:
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self) ->None:
        super().__init__()
        self.data = pd.read_csv('ChnSentiCorp_htl_all.csv')
        self.data = self.data.dropna()
        
    def __getitem__(self, index):
        return self.data.iloc[index]['review'],self.data.iloc[index]['label']
    def __len__(self):
        return len(self.data)
    

In [8]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])


('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


划分数据集

In [9]:
from torch.utils.data import random_split

train_set, valid_set = random_split(dataset, lengths=[.9, .1])
len(train_set),len(valid_set)

(6989, 776)

In [10]:
for i in range(10):
    print(train_set[i])


('欧式简约装潢，有飘窗，可惜周围全是楼，大床是2张小床拼成的，可以看出小床很小，羽绒被、枕头，在跑马地住宅区，比较安静，周边超市、洗衣店多，餐厅一般，出门不易打车，有免费bus到时代广场等地，有收费机场穿梭巴士', 1)
('硬件设施还可以，服务很‘好’，一个晚上竟然打了好几个电话上来问需不需要小姐服务。。。。。。', 0)
('4月中旬，恰逢阴雨连绵，张家界别样的山水风光令人流连忘返。入住湘电酒店：1、房间舒适，一觉到大天亮；2、离景区非常近，凭《度假休闲客人证》可多次出入景区；3、免费机场接机，省心省事；4、大堂周经理等人的服务态度一流，有宾至如归的感觉。其它：1、宾馆部分尚在装修，稍有杂乱地方；2、记得带上两张一寸照片，做《度假休闲客人证》用。', 1)
('非常的好！我和我老婆对这家客栈赞不绝口。基本上可以达到四星级的房屋设施标准。自带的庭院非常美丽。酒店的服务员和老板人都很好。早饭非常好吃，牛奶，馒头，鸡蛋，和肉酱面。真的让人有家的感觉。下次我一定还要住在那里。', 1)
('我是10月5日到万豪住了一个晚上,说真的绝对不能和五星级酒店联系在一起.服务意识确实不错,但晚上以后好象酒店自己有一个象是夜总会的地方,音乐响到凌晨2点多,嘈死人了.以后再也不去住了.', 1)
('标间面积实在太小，卫生间都转不开身。客房服务很一般，好在打电话后态度尚可。值得称赞的是它家的咖啡厅环境不错，中餐厅口味还好。', 0)
('请各位注意酒店提供的饮用水是自来水，这是前台服务员说的。所以各位千万不能生喝酒店提供的饮用水。2008年了，北京的酒店还是这种水平，真是令人心寒。', 0)
('先从停车场开始说起吧，一个保安都没有，停车场是自由出入的，把车停在那里感觉上是停在了路边的空地上，一点安全感没有。进入酒店大堂开始，就有一种刺鼻的霉味，红色地毯脏得都快变成黑色的了。进入房间后的第一感觉就是被别人给扔进了地下室，就这房间也敢叫豪华房？？马上要求退房被拒绝后，酒店主动给我换了个房间，说是更豪华的房间，结果还是，，，，，不过好在窗户可以打开了，我就开着窗户睡了一会，酒店的浴巾居然带有异味（发霉的味道）我真不知道携程是出于什么心态跟这样的酒店签约，出于价格么？？？接近于四百这也叫便宜？？？奉劝大家千万别住这里。唯一可以算的过去的是，服务态度还不错。', 0)
('房间很大,设施很

创建Datalodader

In [21]:
from transformers import AutoModel,AutoConfig
model = AutoModel.from_pretrained("hfl/rbt3", force_download=True)
tokenizer = AutoTokenizer.from_pretrained('rbt3')
def collate_fn(batch):
    texts, labels = [],[]
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    print(texts)
    print(labels)


OSError: rbt3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [19]:
train_loader = DataLoader(dataset=train_set,batch_size=32,shuffle=True,collate_fn=collate_fn)#collate_fn 对batch_size条数据做汇总聚合
valid_loader = DataLoader(dataset=valid_set,batch_size=64,shuffle=False)


In [20]:
next(enumerate(train_loader))[1]

['到敦煌入住这个酒店吃住行都方便，适合驴友。', '周边环境还是不错的，但是酒店的服务人员态度奇差，房间比较小，是我住过酒店中间服务最差的一个。', '协程价格比较好。下次出差还会住这里。房间还不错。', '隔音太差，好像开着窗户一样。吵的没法睡。再出差不敢住这里了房间设施还可以，床上被褥很干净', '非常不错，服务很好，位于市中心区，交通方便，不过价格也高！', '房间较旧，墙纸有霉变。环境一般，感觉不象携程上介绍的那样古色古香，而是感觉破旧不堪。不过周围交通比较方便，离景点和汽车站都很近。', '酒店位置好，环境可以，房间就是小了点，不过出差住在这里是很不错的选择。', '实在忍无可忍。1、水。缺水现象――洗澡至中途，突然断水，望着满身的肥皂泡欲哭无泪；多水现象――有幸目睹了水漫金山的壮观景象。我们住在2楼，半夜，只听到哗哗的水声，之后从房顶上渗出的水，蔓延了整个房间，我们淌着水冲进房间，只见被子已全部湿透，庆幸的是放行李的地方还未受灾，赶紧进行抢救行动。2、安全。大概山上已经提早进入了共产主义社会，客房有门等于没门，服务员可以拿着房卡随意进入任何已经入住客人的客房。半夜，我们已经入睡，可是突然有人开门进来，当我惊醒但还处于迷茫状态的时候，只见服务员在我的房间内若无其事的巡视。我感到自己的人身安全受到了严重的威胁。就这样被折腾了一夜，一宿未睡啊！现在是头昏脑涨、鼻涕一把、心有余悸阿！总结，千万别住在这样的度假村内了，黑店阿。', '还不错的一家酒店，位置比价安静，房间也还不错，就是出门打车要走出一段距离。酒店商品部的服务很一般，态度极其冷淡，应该下岗。', '中规中矩的五星级酒店，但酒店双人房床较小，其他还可以，毕竟也是连锁酒店。另外，酒店的大堂确实有些气势！', '从04年到06年，常带同事和家人住在主楼1号楼。此次CHECK－IN时，被告知住在2号楼，2号楼没有电梯，带着3岁的小孩很麻烦，另外窗外不象1号楼有风景、房间设置陈旧，价格却与1号楼一样，有一种上当的感觉，将来如果不能事先确认，不打算住在这个酒店了。', '总体来讲，对登封的印象比较差，基本上是脏乱差的典型！酒店的设施极端简陋，洗手间里连浴帘都没有，洗个澡满屋子的水！而且，没有挂衣服的地方。酒店里的餐厅，竟然不到7点去就已经黑灯瞎火了。总之比较阴暗老旧的酒店，条件很一般。不过，地理位置不错，对面是